In [56]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Input, Embedding, LSTM, Dense

import pandas as pd
import numpy as np

import pickle
from tqdm import tqdm
import os

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [57]:
data=pd.read_csv("fully_processed_data.csv", nrows=1000)
data.head(2)

,Unnamed: 0,school_state,teacher_prefix,project_grade_category,teacher_number_of_previously_posted_projects,project_is_approved,clean_categories,clean_subcategories,essay,project_title,price,std_price,nrm_price,presence_of_the_numerical_digits,quantity
0,0,in,mrs,grades_prek_2,0,0,literacy_language,esl_literacy,students english learners working english seco...,educational support english learners home,154.6,-0.390533,0.015397,0,23
1,1,fl,mr,grades_6_8,7,1,history_civics_health_sports,civics_government_teamsports,students arrive school eager learn polite gene...,wanted projector hungry learners,299.0,0.002396,0.029839,0,1


## Total Text Data

In [3]:
docs_essay=list(data.essay.values)
labels=np.array(data.project_is_approved.values)

In [5]:
#Prepare tokenizer
tokens = Tokenizer()
tokens.fit_on_texts(docs_essay)
vocab_size = len(tokens.word_index) + 1
#Integer encode the documents
encoded_docs_essay = tokens.texts_to_sequences(docs_essay)
#print(encoded_docs)
print(vocab_size)

9049


In [7]:
max_len=0
all_lengths=[]
for sent in docs_essay:
    length=len(sent.split())
    all_lengths.append(length)
print(max(all_lengths))

303


In [9]:
# pad documents to a max length of 4 words
max_length = max(all_lengths)
padded_docs_essay = pad_sequences(encoded_docs_essay, maxlen=max_length, padding='post')
print(padded_docs_essay)

[[   1   84   45 ...    0    0    0]
 [   1 1384    2 ...    0    0    0]
 [ 590 5400    5 ...    0    0    0]
 ...
 [  76  610  115 ...    0    0    0]
 [   1    3  835 ...    0    0    0]
 [1550 5387 1028 ...    0    0    0]]


In [10]:
#Load the whole embedding into memory
embeddings_index = dict()
file = open('glove.6B.300d.txt')
for line in file:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
file.close()

print('Loaded %s word vectors.' % len(embeddings_index))

#Create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 300))
for word, i in tqdm(tokens.word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector #embedding_matrix.shape: (9049, 300)
        
print(len(embedding_matrix))
print(len(embedding_matrix[0]))

100%|██████████| 9048/9048 [00:00<00:00, 246438.58it/s]

Loaded 400000 word vectors.
9049
300


In [12]:
#Get the flattened LSTM output for input text
input_layer = Input(shape=(max_length,))
embedding = Embedding(input_dim=vocab_size, output_dim=300, weights=[embedding_matrix], input_length=max_length, trainable=False)(input_layer)
lstm_out = LSTM(32, return_sequences=True)(embedding)
flatten_lstm_out = Flatten()(lstm_out)

## Categorical data: school_state 

In [13]:
docs_school_state=list(data.school_state.values)

In [15]:
#Prepare tokenizer
tokens = Tokenizer()
tokens.fit_on_texts(docs_school_state)
vocab_size = len(tokens.word_index) + 1
#Integer encode the documents
encoded_docs_school_state = tokens.texts_to_sequences(docs_school_state)
#print(encoded_docs)
print(vocab_size)

49


In [16]:
max_len=0
all_lengths=[]
for sent in docs_school_state:
    length=len(sent.split())
    all_lengths.append(length)
print(max(all_lengths))

1


In [19]:
# pad documents to a max length of 4 words
max_length = max(all_lengths)
padded_docs_school_state = pad_sequences(encoded_docs_school_state, maxlen=max_length, padding='post')
print(padded_docs_school_state)

[[21]
 [ 3]
 [18]
 [31]
 [ 2]
 [ 3]
 [17]
 [ 7]
 [ 5]
 [ 6]
 [ 1]
 [ 1]
 [ 4]
 [13]
 [22]
 [ 2]
 [ 3]
 [27]
 [ 7]
 [10]
 [ 9]
 [ 6]
 [ 1]
 [19]
 [ 3]
 [19]
 [ 2]
 [20]
 [ 7]
 [11]
 [21]
 [ 6]
 [ 6]
 [25]
 [ 1]
 [ 4]
 [12]
 [ 2]
 [ 1]
 [ 3]
 [ 1]
 [13]
 [37]
 [27]
 [20]
 [33]
 [ 2]
 [23]
 [17]
 [ 1]
 [ 6]
 [ 7]
 [ 4]
 [22]
 [ 4]
 [ 1]
 [31]
 [ 1]
 [25]
 [19]
 [17]
 [ 5]
 [12]
 [ 9]
 [ 3]
 [ 9]
 [ 2]
 [ 3]
 [ 3]
 [ 1]
 [ 1]
 [11]
 [ 2]
 [ 1]
 [32]
 [ 5]
 [11]
 [18]
 [ 5]
 [ 2]
 [23]
 [ 1]
 [23]
 [24]
 [28]
 [ 3]
 [27]
 [ 8]
 [ 1]
 [15]
 [ 3]
 [10]
 [39]
 [ 9]
 [34]
 [23]
 [ 7]
 [41]
 [25]
 [ 1]
 [ 1]
 [ 2]
 [ 1]
 [ 7]
 [ 3]
 [ 8]
 [20]
 [ 5]
 [18]
 [ 4]
 [20]
 [ 4]
 [16]
 [20]
 [ 7]
 [15]
 [10]
 [ 5]
 [ 2]
 [ 1]
 [ 3]
 [ 1]
 [ 2]
 [22]
 [ 6]
 [20]
 [ 2]
 [14]
 [15]
 [ 1]
 [ 9]
 [12]
 [ 2]
 [ 1]
 [ 6]
 [ 2]
 [ 3]
 [ 5]
 [ 2]
 [12]
 [32]
 [ 1]
 [42]
 [23]
 [ 4]
 [ 3]
 [ 3]
 [ 9]
 [30]
 [ 3]
 [24]
 [ 2]
 [ 1]
 [18]
 [ 3]
 [21]
 [ 1]
 [ 4]
 [ 6]
 [10]
 [ 1]
 [23]
 [ 7]
 [15]
 [ 7]
 [ 7]
 [10

In [20]:
#Get the flattened LSTM output for input text
input_layer = Input(shape=(max_length,))
embedding = Embedding(input_dim=vocab_size, output_dim=5, input_length=max_length, trainable=True)(input_layer)
flatten_school_state = Flatten()(embedding)

## Categorical data: project_grade_category 

In [21]:
docs_project_grade_category=list(data.project_grade_category.values)

In [23]:
#Prepare tokenizer
tokens = Tokenizer()
tokens.fit_on_texts(docs_project_grade_category)
vocab_size = len(tokens.word_index) + 1
#Integer encode the documents
encoded_docs_project_grade_category = tokens.texts_to_sequences(docs_project_grade_category)
#print(encoded_docs)
print(vocab_size)

10


In [24]:
encoded_docs_project_grade_category

[[1, 2, 3],
 [1, 6, 7],
 [1, 6, 7],
 [1, 2, 3],
 [1, 2, 3],
 [1, 4, 5],
 [1, 6, 7],
 [1, 4, 5],
 [1, 2, 3],
 [1, 2, 3],
 [1, 4, 5],
 [1, 2, 3],
 [1, 6, 7],
 [1, 2, 3],
 [1, 2, 3],
 [1, 4, 5],
 [1, 2, 3],
 [1, 4, 5],
 [1, 2, 3],
 [1, 4, 5],
 [1, 2, 3],
 [1, 6, 7],
 [1, 8, 9],
 [1, 6, 7],
 [1, 2, 3],
 [1, 4, 5],
 [1, 2, 3],
 [1, 4, 5],
 [1, 8, 9],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 4, 5],
 [1, 6, 7],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 8, 9],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 4, 5],
 [1, 2, 3],
 [1, 2, 3],
 [1, 6, 7],
 [1, 4, 5],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 4, 5],
 [1, 6, 7],
 [1, 2, 3],
 [1, 2, 3],
 [1, 4, 5],
 [1, 6, 7],
 [1, 4, 5],
 [1, 2, 3],
 [1, 6, 7],
 [1, 6, 7],
 [1, 2, 3],
 [1, 2, 3],
 [1, 8, 9],
 [1, 6, 7],
 [1, 4, 5],
 [1, 2, 3],
 [1, 4, 5],
 [1, 4, 5],
 [1, 2, 3],
 [1, 6, 7],
 [1, 4, 5],
 [1, 6, 7],
 [1, 8, 9],
 [1, 4, 5],
 [1, 2, 3],
 [1, 4, 5],
 [1, 4, 5],
 [1, 6, 7],
 [1, 4, 5],
 [1, 4, 5],
 [1, 6, 7],
 [1, 2, 3],
 [1,

In [40]:
max_len=0
all_lengths=[]
for sent in encoded_docs_project_grade_category:
    length=len(sent)
    all_lengths.append(length)
print(max(all_lengths))

3


In [41]:
# pad documents to a max length of 4 words
max_length = max(all_lengths)
padded_docs_project_grade_category = pad_sequences(encoded_docs_project_grade_category, maxlen=max_length, padding='post')
print(padded_docs_project_grade_category)

[[1 2 3]
 [1 6 7]
 [1 6 7]
 ...
 [1 2 3]
 [1 2 3]
 [1 2 3]]


In [28]:
#Get the flattened LSTM output for input text
input_layer = Input(shape=(max_length,))
embedding = Embedding(input_dim=vocab_size, output_dim=5, input_length=max_length, trainable=True)(input_layer)
flatten_project_grade_category = Flatten()(embedding)

## Categorical data: clean_categories

In [30]:
docs_clean_categories=list(data.clean_categories.values)

In [31]:
#Prepare tokenizer
tokens = Tokenizer()
tokens.fit_on_texts(docs_clean_categories)
vocab_size = len(tokens.word_index) + 1
#Integer encode the documents
encoded_docs_clean_categories = tokens.texts_to_sequences(docs_clean_categories)
#print(encoded_docs)
print(vocab_size)

16


In [32]:
encoded_docs_clean_categories

[[1, 2],
 [11, 12, 5, 6],
 [5, 6],
 [1, 2, 3, 4],
 [3, 4],
 [1, 2, 7],
 [1, 2, 7],
 [3, 4],
 [5, 6],
 [1, 2],
 [1, 2],
 [1, 2, 8],
 [3, 4],
 [7],
 [1, 2],
 [5, 6],
 [1, 2, 7],
 [3, 4, 1, 2],
 [8],
 [5, 6],
 [1, 2],
 [3, 4, 7],
 [1, 2],
 [9, 10],
 [3, 4],
 [3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 7],
 [1, 2, 8],
 [1, 2],
 [7],
 [3, 4, 1, 2],
 [11, 12],
 [1, 2],
 [5, 6],
 [1, 2, 3, 4],
 [5, 6, 1, 2],
 [5, 6],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2, 9, 10],
 [3, 4],
 [1, 2],
 [1, 2],
 [13, 14, 15],
 [1, 2, 3, 4],
 [5, 6],
 [5, 6],
 [1, 2],
 [3, 4, 11, 12],
 [1, 2],
 [5, 6],
 [3, 4],
 [7],
 [1, 2, 3, 4],
 [1, 2],
 [1, 2],
 [5, 6],
 [3, 4],
 [1, 2],
 [9, 10],
 [9, 10],
 [7],
 [3, 4],
 [1, 2],
 [3, 4],
 [8, 1, 2],
 [3, 4],
 [3, 4],
 [1, 2],
 [8],
 [3, 4],
 [9, 10],
 [1, 2, 3, 4],
 [8, 7],
 [3, 4],
 [9, 10],
 [1, 2],
 [3, 4],
 [1, 2],
 [1, 2, 11, 12],
 [3, 4],
 [1, 2],
 [5, 6],
 [1, 2, 7],
 [7],
 [1, 2],
 [1, 2, 9, 10],
 [5, 6],
 [8, 5, 6],
 [9, 10],
 [1, 2],
 [3, 4],
 [1, 2],
 [1, 2

In [36]:
max_len=0
all_lengths=[]
for sent in encoded_docs_clean_categories:
    length=len(sent)
    all_lengths.append(length)
print(max(all_lengths))

5


In [38]:
# pad documents to a max length of 4 words
max_length = max(all_lengths)
padded_docs_clean_categories = pad_sequences(encoded_docs_clean_categories, maxlen=max_length, padding='post')
print(padded_docs_clean_categories)

[[ 1  2  0  0  0]
 [11 12  5  6  0]
 [ 5  6  0  0  0]
 ...
 [ 1  2  7  0  0]
 [ 1  2  3  4  0]
 [ 1  2  7  0  0]]


In [28]:
#Get the flattened LSTM output for input text
input_layer = Input(shape=(max_length,))
embedding = Embedding(input_dim=vocab_size, output_dim=5, input_length=max_length, trainable=True)(input_layer)
flatten_clean_categories = Flatten()(embedding)

## Categorical data: clean_subcategories

In [42]:
docs_clean_subcategories=list(data.clean_subcategories.values)

In [43]:
#Prepare tokenizer
tokens = Tokenizer()
tokens.fit_on_texts(docs_clean_subcategories)
vocab_size = len(tokens.word_index) + 1
#Integer encode the documents
encoded_docs_clean_subcategories = tokens.texts_to_sequences(docs_clean_subcategories)
#print(encoded_docs)
print(vocab_size)

38


In [44]:
encoded_docs_clean_subcategories

[[16, 1],
 [30, 31, 23],
 [5, 8, 23],
 [1, 2],
 [2],
 [3, 4, 6],
 [1, 6],
 [2],
 [5, 8],
 [1, 3, 4],
 [1],
 [1, 32],
 [9, 5, 15],
 [6],
 [1],
 [5, 8],
 [1, 6],
 [7, 3, 4],
 [14],
 [5, 8],
 [1],
 [5, 15, 6],
 [1],
 [13],
 [7, 2],
 [2],
 [1, 2],
 [34, 2],
 [1, 6],
 [1, 17],
 [1],
 [6],
 [5, 15, 1],
 [35, 36],
 [3, 4],
 [23],
 [3, 4, 2],
 [5, 8, 1],
 [11, 12, 5, 8],
 [1, 3, 4],
 [1, 3, 4],
 [3, 4],
 [1, 3, 4],
 [1, 10],
 [2],
 [1, 3, 4],
 [3, 4],
 [26, 27, 28],
 [3, 4, 2],
 [11, 12, 5, 8],
 [11, 12, 5, 8],
 [1],
 [2, 25],
 [1],
 [11, 12, 5, 8],
 [7, 2],
 [6],
 [3, 4, 2],
 [3, 4],
 [1, 3, 4],
 [5, 8],
 [7, 2],
 [3, 4],
 [10],
 [10],
 [6],
 [7, 2],
 [1],
 [7, 2],
 [18, 19, 3, 4],
 [9],
 [7, 5, 15],
 [16, 3, 4],
 [18, 19],
 [5, 15, 2],
 [13],
 [3, 4, 2],
 [24, 6],
 [2],
 [13],
 [1, 3, 4],
 [9, 2],
 [1],
 [1, 25],
 [7, 2],
 [3, 4],
 [5, 8, 23],
 [1, 6],
 [6],
 [1],
 [3, 4, 10],
 [5, 8],
 [24, 5, 8],
 [13, 20],
 [1],
 [2],
 [1],
 [1],
 [33],
 [2],
 [3, 4, 2],
 [11, 12, 23],
 [1, 2],
 [13, 20],

In [45]:
max_len=0
all_lengths=[]
for sent in encoded_docs_clean_subcategories:
    length=len(sent)
    all_lengths.append(length)
print(max(all_lengths))

4


In [46]:
# pad documents to a max length of 4 words
max_length = max(all_lengths)
padded_docs_clean_subcategories = pad_sequences(encoded_docs_clean_subcategories, maxlen=max_length, padding='post')
print(padded_docs_clean_subcategories)

[[16  1  0  0]
 [30 31 23  0]
 [ 5  8 23  0]
 ...
 [ 1  6  0  0]
 [ 3  4  2  0]
 [16  6  0  0]]


In [47]:
#Get the flattened LSTM output for input text
input_layer = Input(shape=(max_length,))
embedding = Embedding(input_dim=vocab_size, output_dim=5, input_length=max_length, trainable=True)(input_layer)
flatten_clean_subcategories = Flatten()(embedding)

## Categorical data: teacher_prefix

In [48]:
docs_teacher_prefix=list(data.teacher_prefix.values)

In [49]:
#Prepare tokenizer
tokens = Tokenizer()
tokens.fit_on_texts(docs_teacher_prefix)
vocab_size = len(tokens.word_index) + 1
#Integer encode the documents
encoded_docs_teacher_prefix = tokens.texts_to_sequences(docs_teacher_prefix)
#print(encoded_docs)
print(vocab_size)

5


In [50]:
encoded_docs_teacher_prefix

[[1],
 [3],
 [2],
 [1],
 [1],
 [1],
 [1],
 [2],
 [1],
 [2],
 [1],
 [2],
 [1],
 [1],
 [2],
 [2],
 [1],
 [2],
 [1],
 [2],
 [1],
 [1],
 [2],
 [3],
 [1],
 [1],
 [2],
 [4],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [2],
 [3],
 [1],
 [2],
 [1],
 [2],
 [2],
 [1],
 [2],
 [1],
 [1],
 [1],
 [2],
 [1],
 [2],
 [1],
 [1],
 [2],
 [3],
 [2],
 [2],
 [1],
 [2],
 [1],
 [1],
 [3],
 [2],
 [1],
 [3],
 [3],
 [2],
 [2],
 [1],
 [2],
 [2],
 [2],
 [1],
 [1],
 [2],
 [1],
 [1],
 [4],
 [2],
 [1],
 [1],
 [2],
 [2],
 [1],
 [4],
 [2],
 [1],
 [2],
 [2],
 [2],
 [4],
 [2],
 [1],
 [1],
 [2],
 [2],
 [1],
 [2],
 [2],
 [1],
 [1],
 [1],
 [2],
 [1],
 [2],
 [3],
 [2],
 [3],
 [1],
 [1],
 [1],
 [2],
 [2],
 [1],
 [1],
 [1],
 [1],
 [1],
 [2],
 [1],
 [1],
 [1],
 [1],
 [2],
 [2],
 [1],
 [1],
 [1],
 [1],
 [1],
 [3],
 [1],
 [1],
 [1],
 [2],
 [1],
 [1],
 [3],
 [2],
 [1],
 [1],
 [1],
 [2],
 [2],
 [2],
 [1],
 [1],
 [1],
 [1],
 [1],
 [2],
 [1],
 [2],
 [2],
 [1],
 [1],
 [2],
 [1],
 [2],
 [1],
 [1],
 [3],
 [1],
 [1],
 [2],
 [2],
 [3],
 [2],
 [3]

In [51]:
max_len=0
all_lengths=[]
for sent in encoded_docs_teacher_prefix:
    length=len(sent)
    all_lengths.append(length)
print(max(all_lengths))

1


In [52]:
# pad documents to a max length of 4 words
max_length = max(all_lengths)
padded_docs_teacher_prefix = pad_sequences(encoded_docs_teacher_prefix, maxlen=max_length, padding='post')
print(padded_docs_teacher_prefix)

[[1]
 [3]
 [2]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [1]
 [2]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [1]
 [2]
 [3]
 [1]
 [1]
 [2]
 [4]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [3]
 [1]
 [2]
 [1]
 [2]
 [2]
 [1]
 [2]
 [1]
 [1]
 [1]
 [2]
 [1]
 [2]
 [1]
 [1]
 [2]
 [3]
 [2]
 [2]
 [1]
 [2]
 [1]
 [1]
 [3]
 [2]
 [1]
 [3]
 [3]
 [2]
 [2]
 [1]
 [2]
 [2]
 [2]
 [1]
 [1]
 [2]
 [1]
 [1]
 [4]
 [2]
 [1]
 [1]
 [2]
 [2]
 [1]
 [4]
 [2]
 [1]
 [2]
 [2]
 [2]
 [4]
 [2]
 [1]
 [1]
 [2]
 [2]
 [1]
 [2]
 [2]
 [1]
 [1]
 [1]
 [2]
 [1]
 [2]
 [3]
 [2]
 [3]
 [1]
 [1]
 [1]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [3]
 [2]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [2]
 [2]
 [1]
 [1]
 [2]
 [1]
 [2]
 [1]
 [1]
 [3]
 [1]
 [1]
 [2]
 [2]
 [3]
 [2]
 [3]
 [3]
 [3]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [1]
 [2]
 [1]
 [3]
 [2]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [3]
 [1]
 [2]
 [1]
 [3]
 [1]
 [1]
 [1]


In [53]:
#Get the flattened LSTM output for input text
input_layer = Input(shape=(max_length,))
embedding = Embedding(input_dim=vocab_size, output_dim=5, input_length=max_length, trainable=True)(input_layer)
flatten_clean_subcategories = Flatten()(embedding)

## teacher_number_of_previously_posted_projects, nrm_price, presence_of_the_numerical_digits

In [59]:
teacher_number_of_previously_posted_projects=list(data.teacher_number_of_previously_posted_projects.values)
presence_of_the_numerical_digits=list(data.presence_of_the_numerical_digits.values)
nrm_price=list(data.nrm_price.values)

In [61]:
len(teacher_number_of_previously_posted_projects)

1000

In [49]:
#Prepare tokenizer
tokens = Tokenizer()
tokens.fit_on_texts(docs_teacher_prefix)
vocab_size = len(tokens.word_index) + 1
#Integer encode the documents
encoded_docs_teacher_prefix = tokens.texts_to_sequences(docs_teacher_prefix)
#print(encoded_docs)
print(vocab_size)

5


In [50]:
encoded_docs_teacher_prefix

[[1],
 [3],
 [2],
 [1],
 [1],
 [1],
 [1],
 [2],
 [1],
 [2],
 [1],
 [2],
 [1],
 [1],
 [2],
 [2],
 [1],
 [2],
 [1],
 [2],
 [1],
 [1],
 [2],
 [3],
 [1],
 [1],
 [2],
 [4],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [2],
 [3],
 [1],
 [2],
 [1],
 [2],
 [2],
 [1],
 [2],
 [1],
 [1],
 [1],
 [2],
 [1],
 [2],
 [1],
 [1],
 [2],
 [3],
 [2],
 [2],
 [1],
 [2],
 [1],
 [1],
 [3],
 [2],
 [1],
 [3],
 [3],
 [2],
 [2],
 [1],
 [2],
 [2],
 [2],
 [1],
 [1],
 [2],
 [1],
 [1],
 [4],
 [2],
 [1],
 [1],
 [2],
 [2],
 [1],
 [4],
 [2],
 [1],
 [2],
 [2],
 [2],
 [4],
 [2],
 [1],
 [1],
 [2],
 [2],
 [1],
 [2],
 [2],
 [1],
 [1],
 [1],
 [2],
 [1],
 [2],
 [3],
 [2],
 [3],
 [1],
 [1],
 [1],
 [2],
 [2],
 [1],
 [1],
 [1],
 [1],
 [1],
 [2],
 [1],
 [1],
 [1],
 [1],
 [2],
 [2],
 [1],
 [1],
 [1],
 [1],
 [1],
 [3],
 [1],
 [1],
 [1],
 [2],
 [1],
 [1],
 [3],
 [2],
 [1],
 [1],
 [1],
 [2],
 [2],
 [2],
 [1],
 [1],
 [1],
 [1],
 [1],
 [2],
 [1],
 [2],
 [2],
 [1],
 [1],
 [2],
 [1],
 [2],
 [1],
 [1],
 [3],
 [1],
 [1],
 [2],
 [2],
 [3],
 [2],
 [3]

In [51]:
max_len=0
all_lengths=[]
for sent in encoded_docs_teacher_prefix:
    length=len(sent)
    all_lengths.append(length)
print(max(all_lengths))

1


In [52]:
# pad documents to a max length of 4 words
max_length = max(all_lengths)
padded_docs_teacher_prefix = pad_sequences(encoded_docs_teacher_prefix, maxlen=max_length, padding='post')
print(padded_docs_teacher_prefix)

[[1]
 [3]
 [2]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [1]
 [2]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [1]
 [2]
 [3]
 [1]
 [1]
 [2]
 [4]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [3]
 [1]
 [2]
 [1]
 [2]
 [2]
 [1]
 [2]
 [1]
 [1]
 [1]
 [2]
 [1]
 [2]
 [1]
 [1]
 [2]
 [3]
 [2]
 [2]
 [1]
 [2]
 [1]
 [1]
 [3]
 [2]
 [1]
 [3]
 [3]
 [2]
 [2]
 [1]
 [2]
 [2]
 [2]
 [1]
 [1]
 [2]
 [1]
 [1]
 [4]
 [2]
 [1]
 [1]
 [2]
 [2]
 [1]
 [4]
 [2]
 [1]
 [2]
 [2]
 [2]
 [4]
 [2]
 [1]
 [1]
 [2]
 [2]
 [1]
 [2]
 [2]
 [1]
 [1]
 [1]
 [2]
 [1]
 [2]
 [3]
 [2]
 [3]
 [1]
 [1]
 [1]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [3]
 [2]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [2]
 [2]
 [1]
 [1]
 [2]
 [1]
 [2]
 [1]
 [1]
 [3]
 [1]
 [1]
 [2]
 [2]
 [3]
 [2]
 [3]
 [3]
 [3]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [1]
 [2]
 [1]
 [3]
 [2]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [3]
 [1]
 [2]
 [1]
 [3]
 [1]
 [1]
 [1]


In [53]:
#Get the flattened LSTM output for input text
input_layer = Input(shape=(max_length,))
embedding = Embedding(input_dim=vocab_size, output_dim=5, input_length=max_length, trainable=True)(input_layer)
flatten_clean_subcategories = Flatten()(embedding)

In [12]:
# define model
model = Sequential()
e = Embedding(input_dim=vocab_size, output_dim=300, weights=[embedding_matrix], input_length=max_length, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 303, 300)          2714700   
_________________________________________________________________
flatten_2 (Flatten)          (None, 90900)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 90901     
Total params: 2,805,601
Trainable params: 90,901
Non-trainable params: 2,714,700
_________________________________________________________________
None
Accuracy: 100.000000


In [24]:
import pdfkit
pdfkit.from_file('Test Pad.html', 'Test Pad.pdf')

Loading page (1/2)
Printing pages (2/2)                                               
Done                                                           


True

In [35]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

[[10, 16], [31, 6], [36, 40], [27, 6], [9], [18], [48, 40], [1, 31], [48, 6], [14, 31, 16, 42]]
[[10 16  0  0]
 [31  6  0  0]
 [36 40  0  0]
 [27  6  0  0]
 [ 9  0  0  0]
 [18  0  0  0]
 [48 40  0  0]
 [ 1 31  0  0]
 [48  6  0  0]
 [14 31 16 42]]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_8 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None
Accuracy: 100.000000


In [37]:
len(encoded_docs)

10